   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [35]:
# 2.71 - 1.13

In [13]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [17]:
data = pd.read_excel("komentar/data_cleanU1.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.tail()

,Column1,Column2,Column3,Prediksi,label,komentar
5042,5049,NaN,NaN,spam,spam,bosen outfit kalian aja yuk cek ig tambah kole...
5043,5050,NaN,NaN,spam,spam,maantaap banggeet kaak eteaah akuu pakee gemuk...
5044,5052,NaN,NaN,spam,spam,sllu ajar usaha tahan kualitas costumer neng s...
5045,5053,NaN,NaN,spam,spam,aasi ane o a uda nenain odu yan sana eui eecay...
5046,5054,NaN,NaN,spam,spam,awal ny ga percaya aku coba nyata hasil cepet ...


# Pemodelan

## TF-IDF

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(5047, 15136)


## Pemodelan MNB 

In [19]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Pemodelan dan Validasi Model

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
# print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #latih model
        clf.fit(X_train,y_train)
        
        #prediksi
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
#     print('alpha   :',alpha)
#     print('     akurasi :',rerata_akurasi)
#     print("="*100)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................


,alpha,akurasi
0,0.001,0.961976
1,0.002,0.963758
2,0.003,0.963758
3,0.004,0.964551
4,0.005,0.964947
5,0.006,0.964947
6,0.007,0.964947
7,0.008,0.964947
8,0.009,0.965145
9,0.010,0.965541


In [21]:
print(alpha_list[index])
print(akurasi[index])

0.402
0.9695091807317378


In [9]:
print(alpha_list[index])
print(akurasi[index])

0.296
0.9268460858027421


# Menyimpan Model

In [12]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)
dump(clf, "model/mnb.m")

['model/mnb.m']